# **🚌 Tugas Lab 1**

1. Buatlah model klasifikasi dengan menggunakan kNN untuk mengklasifikasikan jenis suara `male` dan `female` pada dataset `voice.csv`.

2. Lakukan percobaan untuk mengetahui fitur-fitur yang paling optimal untuk digunakan. Fitur apa saja yang Anda gunakan untuk mendapatkan hasil terbaik?

3. Berdasarkan fitur yang telah Anda pilih pada soal nomor 2, berapa nilai `k` yang terbaik? Lampirkan grafika analisis dan alasan Anda.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# --- 0. Pemuatan dan Persiapan Data ---
try:
    df = pd.read_csv('voice.csv')
except FileNotFoundError:
    print("Error: File 'voice.csv' tidak ditemukan.")
    print("Pastikan file tersebut berada di direktori yang sama dengan skrip ini.")

print("--- Informasi Data Awal ---")
print(df.info())
print("\nContoh Data:")
print(df.head())

In [ ]:
# Konversi label 'male'/'female' menjadi numerik (1/0)
# Ini penting untuk analisis korelasi
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])
# le.classes_ akan menunjukkan ['female', 'male'], artinya female=0, male=1

# Pisahkan fitur (X) dan target (y)
# Kita gunakan label_encoded sebagai target
X = df.drop(columns=['label', 'label_encoded'])
y = df['label_encoded']

In [ ]:
# --- TUGAS 1: Model Klasifikasi kNN (Baseline) ---
print("\n--- TUGAS 1: Model Baseline (Semua Fitur) ---")

# Bagi data menjadi training and testing
# stratify=y memastikan proporsi male/female sama di train dan test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# kNN sensitif terhadap skala fitur, jadi kita perlu StandardScaler
scaler_all = StandardScaler()
X_train_scaled = scaler_all.fit_transform(X_train)
X_test_scaled = scaler_all.transform(X_test)

# Gunakan k=5 sebagai baseline awal
knn_baseline = KNeighborsClassifier(n_neighbors=5)
knn_baseline.fit(X_train_scaled, y_train)
y_pred_baseline = knn_baseline.predict(X_test_scaled)
acc_baseline = accuracy_score(y_test, y_pred_baseline)

print(f"Akurasi Model Baseline (Semua Fitur, k=5): {acc_baseline:.4f}")
print("Laporan Klasifikasi Baseline:")
# target_names=['female', 'male'] didapat dari le.classes_
print(classification_report(y_test, y_pred_baseline, target_names=['female', 'male']))

In [ ]:
# --- TUGAS 2: Analisis dan Pemilihan Fitur Optimal ---
print("\n--- TUGAS 2: Analisis Fitur Optimal ---")

# Hitung korelasi absolut antara setiap fitur dengan target ('label_encoded')
corr_matrix = df.drop(columns=['label']).corr()
corr_target = corr_matrix['label_encoded'].abs().sort_values(ascending=False)

print("Korelasi Absolut Fitur dengan Target (terurut):")
# Tampilkan semua fitur, kecuali 'label_encoded' itu sendiri
print(corr_target[1:])

# Mari kita pilih N fitur teratas.
# Berdasarkan output di atas, 'meanfun' (frekuensi fundamental rata-rata)
# dan 'IQR' (Interquartile Range) biasanya sangat kuat.

# Kita akan coba gunakan 4 fitur teratas
# Skrip ini memilihnya secara dinamis
N_FITUR_TERBAIK = 4
optimal_features = list(corr_target[1:N_FITUR_TERBAIK + 1].index)

print(f"\nFitur yang dipilih sebagai 'optimal' ({N_FITUR_TERBAIK} teratas): {optimal_features}")

# Buat dataset baru hanya dengan fitur optimal
X_opt = df[optimal_features]

# Lakukan train/test split lagi untuk data optimal ini
X_train_opt, X_test_opt, y_train_opt, y_test_opt = train_test_split(
    X_opt, y, test_size=0.2, random_state=42, stratify=y
)

# Scaling dataset optimal
scaler_opt = StandardScaler()
X_train_opt_scaled = scaler_opt.fit_transform(X_train_opt)
X_test_opt_scaled = scaler_opt.transform(X_test_opt)

# Uji model k=5 dengan fitur optimal
knn_opt_k5 = KNeighborsClassifier(n_neighbors=5)
knn_opt_k5.fit(X_train_opt_scaled, y_train_opt)
y_pred_opt_k5 = knn_opt_k5.predict(X_test_opt_scaled)
acc_opt_k5 = accuracy_score(y_test_opt, y_pred_opt_k5)

print(f"Akurasi dengan {N_FITUR_TERBAIK} Fitur Optimal (k=5): {acc_opt_k5:.4f}")
print(f"\nPerbandingan: Baseline ({acc_baseline:.4f}) vs Optimal ({acc_opt_k5:.4f})")
print("Hasil ini menunjukkan bahwa dengan menggunakan lebih sedikit fitur (yang paling berkorelasi),")
print("kita bisa mendapatkan akurasi yang sebanding atau bahkan lebih baik,")
print("membuat model lebih efisien.")

In [ ]:
# --- TUGAS 3: Mencari Nilai k Terbaik (dengan Fitur Optimal) ---
print("\n--- TUGAS 3: Analisis Nilai k Terbaik ---")

k_range = range(1, 31) # Kita akan uji k dari 1 sampai 30
accuracies = []

# Loop untuk menguji setiap nilai k
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_opt_scaled, y_train_opt)
    y_pred = knn.predict(X_test_opt_scaled)
    accuracies.append(accuracy_score(y_test_opt, y_pred))

# Cari k dengan akurasi tertinggi
best_k = k_range[np.argmax(accuracies)]
best_acc = np.max(accuracies)

print(f"\nNilai k terbaik ditemukan: k = {best_k}")
print(f"Akurasi tertinggi yang dicapai: {best_acc:.4f}")

print("\nAlasan Pemilihan:")
print(f"Nilai k={best_k} memberikan akurasi tertinggi pada data tes.")
print("Dari grafik (akan ditampilkan), kita mencari nilai 'k' yang memberikan")
print("akurasi tinggi namun juga stabil (tidak terlalu 'tajam' puncaknya).")
print("Nilai k yang terlalu kecil (misal k=1) sangat rentan terhadap noise (overfitting).")
print("Nilai k yang terlalu besar cenderung 'terlalu halus' (oversmoothing) dan bisa")
print("kehilangan detail-detail penting.")
print(f"Nilai k={best_k} (atau nilai di sekitarnya yang juga tinggi) adalah")
print("pilihan yang baik untuk keseimbangan ini.")

In [ ]:
# --- TUGAS 3 (Lanjutan): Grafika Analisis ---
plt.figure(figsize=(12, 7))
plt.plot(k_range, accuracies, color='blue', linestyle='dashed', marker='o', markerfacecolor='red', markersize=8)
plt.title('Analisis Nilai k: Akurasi vs. Nilai k\n(Menggunakan Fitur Optimal)')
plt.xlabel('Nilai k')
plt.ylabel('Akurasi')
plt.xticks(range(1, 31, 2)) # Tampilkan label k setiap 2 angka
plt.grid(True, linestyle='--', alpha=0.6)

# Tandai nilai k terbaik
plt.axvline(x=best_k, color='green', linestyle=':', label=f'k Terbaik = {best_k} (Akurasi: {best_acc:.4f})')
plt.legend()

print("\nMenampilkan grafik analisis nilai k...")
# Simpan grafik ke file
try:
    plt.savefig('analisis_knn_k.png')
    print("Grafik juga disimpan sebagai 'analisis_knn_k.png'")
except Exception as e:
    print(f"Tidak dapat menyimpan grafik: {e}")

# Tampilkan grafik
plt.show()